## Imports and Functions

We begin by loading the necessary packages/modules ad define a function that merges data pulled from Refinitiv in the [`summary`](<..\data\raw\summary>) file directory and creates two dataframes, one containing all of the merged entries and one containing summary data.

In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime

def process_summary(directory):
    """
    Processes and summarizes financial data from Excel files in a directory.

    Args:
    directory (str): Path to the directory containing the Excel files.

    Returns:
    tuple: Contains two pandas DataFrames. 
        1. main_df (DataFrame): Summary information for each ticker, including average bid, ask, 
           average daily trading volume (ADTV), and historical volatility (Vola).
        2. merged_df (DataFrame): Combined data from all Excel files with an added 'Ticker' 
           column. Filters records between specific dates and reorders columns.
    """
    ticker_data = []
    merged_df = pd.DataFrame()

    # loop through all files in the directory
    for file in os.listdir(directory):
        if file.endswith('.xlsx'):
            file_path = os.path.join(directory, file)

            # read without headers to find the correct header row
            temp_df = pd.read_excel(file_path, header=None)
            temp_df = temp_df.astype(str)  # convert all columns to strings

            header_row = temp_df[temp_df.apply(lambda x: x.str.contains('Exchange Date', na=False)).any(axis=1)].index[0]
            
            # re-read with correct header row
            df = pd.read_excel(file_path, header=header_row)

            # convert 'Exchange Date' to datetime and filter by date range
            df['Exchange Date'] = pd.to_datetime(df['Exchange Date'])
            start_date = datetime(2023, 12, 21)
            end_date = datetime(2024, 3, 18)
            df = df[df['Exchange Date'].between(start_date, end_date)]

            # calc daily returns
            df['Daily Return'] = df['Close'].pct_change()

            # calc historical volatility
            volatility = df['Daily Return'].std()

            # calc avgs
            averages = df[['Close', 'Bid', 'Ask', 'Volume']].mean()

            # append data to list
            ticker = file.split('.')[0]
            ticker_data.append([ticker, averages['Close'], averages['Bid'], averages['Ask'], averages['Volume'], volatility])

            # concatenate to merged_df
            df['Ticker'] = ticker
            merged_df = pd.concat([merged_df, df], ignore_index=True)

    # reorder columns
    cols = ['Ticker'] + [col for col in merged_df if col != 'Ticker']
    merged_df = merged_df[cols]

    # convert list of data to a DataFrame
    main_df = pd.DataFrame(ticker_data, columns=['Ticker', 'Avg Close', 'Avg Bid', 'Avg Ask', 'ADTV', 'Vola'])

    return main_df, merged_df

## Merging Issue-Level Time Series Data

We continue by merging summary statistics and market cap data for our sample period (21.12.2023 to 18.03.2024). Please note that market capitalization is calculated by multiplying number of default shares with price close.

In [2]:
result_df, total_df = process_summary(os.path.join('..', 'data', 'raw', 'summary'))
display(total_df)

C:\Users\Diana\AppData\Local\Temp\ipykernel_1804\1572883446.py:44: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['Daily Return'] = df['Close'].pct_change()


,Ticker,Exchange Date,Close,Adjusted Close,Net,%Chg,Open,Low,High,Volume,Turnover - CHF,Flow,Bid,Ask,Unnamed: 13,Unnamed: 14,Unnamed: 15,Daily Return,Unnamed: 12
0,ABBN,2024-03-18,42.30,NaN,-0.10,-0.002358,42.42,42.19,42.47,3170244.0,134177657.0,2.223399e+09,42.29,42.30,NaN,NaN,NaN,NaN,NaN
1,ABBN,2024-03-15,42.40,NaN,0.09,0.002127,42.27,41.92,42.40,14619323.0,618523232.0,2.357577e+09,42.37,42.40,NaN,NaN,NaN,0.002364,NaN
2,ABBN,2024-03-14,42.31,NaN,0.26,0.006183,42.15,42.09,42.72,5325771.0,225840284.0,1.739053e+09,42.31,42.32,NaN,NaN,NaN,-0.002123,NaN
3,ABBN,2024-03-13,42.05,NaN,0.53,0.012765,41.69,41.50,42.08,4401158.0,184143020.0,1.513213e+09,42.05,42.06,NaN,NaN,NaN,-0.006145,NaN
4,ABBN,2024-03-12,41.52,NaN,0.57,0.013919,41.02,40.69,41.52,4394509.0,180571666.0,1.329070e+09,41.51,41.52,NaN,NaN,NaN,-0.012604,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
880,VLRT,2023-12-29,12.70,NaN,0.00,0.000000,12.70,12.70,12.70,200.0,2540.0,-2.967000e+04,10.50,12.70,NaN,NaN,NaN,0.016000,NaN
881,VLRT,2023-12-28,12.70,NaN,-0.05,-0.003922,12.70,12.70,12.70,1668.0,21184.0,-2.713000e+04,10.50,12.70,NaN,NaN,NaN,0.000000,NaN
882,VLRT,2023-12-27,12.75,NaN,0.30,0.024096,12.75,12.75,12.75,100.0,1275.0,-5.946000e+03,10.50,12.75,NaN,NaN,NaN,0.003937,NaN
883,VLRT,2023-12-22,12.45,NaN,-1.05,-0.077778,12.45,12.45,12.45,580.0,7221.0,-7.221000e+03,10.50,12.45,NaN,NaN,NaN,-0.023529,NaN


We don't bother with creating a custom function to merge MC data. After reading the file, we merge average MC values with our `result_df` dataframe and time series MC values with our `total_df` dataframe with ticker as the key.

In [3]:
mc_df = pd.read_excel(os.path.join('..', 'data', 'raw', 'market_cap.xlsx'))
mc_df['Identifier'] = mc_df['Identifier'].str.replace('.S', '', regex=False)

avg_df = mc_df[['Identifier', 'Company Name', 'Avg MC']].copy()

sum_df = pd.merge(avg_df, result_df, left_on='Identifier', right_on='Ticker')
display(sum_df)

,Identifier,Company Name,Avg MC,Ticker,Avg Close,Avg Bid,Avg Ask,ADTV,Vola
0,ABBN,Abb Ltd,7.224162e+10,ABBN,38.438814,38.430678,38.443390,3.117397e+06,0.011094
1,CLN,Clariant AG,3.790858e+09,CLN,11.337797,11.331864,11.343051,9.777089e+05,0.012418
2,DOKA,Dormakaba Holding AG,1.870050e+09,DOKA,444.389831,444.076271,444.703390,4.409932e+03,0.013400
3,GIVN,Givaudan SA,3.331868e+10,GIVN,3613.237288,3612.220339,3613.694915,1.869014e+04,0.014064
4,KUD,Kudelski SA,6.584905e+07,KUD,1.286949,1.277119,1.304237,6.600285e+04,0.052418
5,LISN,Chocoladefabriken Lindt & Spruengli AG,1.453010e+10,LISN,108298.305085,107949.152542,108413.559322,1.026271e+02,0.012734
6,LONN,Lonza Group AG,3.091884e+10,LONN,418.983051,418.901695,419.050847,2.393532e+05,0.022443
7,NESN,Nestle SA,2.580040e+11,NESN,96.601017,96.595254,96.609492,3.676267e+06,0.011871
8,SCHN,Schindler Holding AG,1.407383e+10,SCHN,210.445763,210.284746,210.516949,2.383359e+04,0.009996
9,SCMN,Swisscom AG,2.644564e+10,SCMN,510.242373,510.150847,510.376271,8.610905e+04,0.009127


We convert the MC dataset from wide format (where each column represents a different date) to long format (where there is a single row for each ticker-date combination) by pivoting the time series columns while keeping the static columns unchanged.

In [4]:
# drop unnecessary MC column
mc_df.drop(['Avg MC'], axis=1, inplace=True)
mc_df.dropna(inplace=True)

# set the static columns
static_columns = ['Identifier', 'Company Name']

# initialize a list to store the new rows
new_rows = []

# pivot
df_melted = mc_df.melt(id_vars=['Identifier', 'Company Name'], var_name='Date', value_name='Market Cap')

# convert 'Date' to datetime
df_melted['Date'] = pd.to_datetime(df_melted['Date'])

# sort by 'Identifier' and 'Date'
df_melted.sort_values(by=['Identifier', 'Date'], inplace=True)

display(df_melted)

,Identifier,Company Name,Date,Market Cap
1320,ABBN,Abb Ltd,2023-12-21,7.017988e+10
1305,ABBN,Abb Ltd,2023-12-22,7.012342e+10
1290,ABBN,Abb Ltd,2023-12-23,7.012342e+10
1275,ABBN,Abb Ltd,2023-12-24,7.012342e+10
1260,ABBN,Abb Ltd,2023-12-25,7.012342e+10
...,...,...,...,...
74,VLRT,Valartis Group AG,2024-03-14,3.726162e+07
59,VLRT,Valartis Group AG,2024-03-15,3.703402e+07
44,VLRT,Valartis Group AG,2024-03-16,3.703402e+07
29,VLRT,Valartis Group AG,2024-03-17,3.703402e+07


In [5]:
total_df = pd.merge(total_df, df_melted, left_on= ['Ticker', 'Exchange Date'], right_on=['Identifier', 'Date'])

display(total_df)

,Ticker,Exchange Date,Close,Adjusted Close,Net,%Chg,Open,Low,High,Volume,...,Ask,Unnamed: 13,Unnamed: 14,Unnamed: 15,Daily Return,Unnamed: 12,Identifier,Company Name,Date,Market Cap
0,ABBN,2024-03-18,42.30,NaN,-0.10,-0.002358,42.42,42.19,42.47,3170244.0,...,42.30,NaN,NaN,NaN,NaN,NaN,ABBN,Abb Ltd,2024-03-18,7.960871e+10
1,ABBN,2024-03-15,42.40,NaN,0.09,0.002127,42.27,41.92,42.40,14619323.0,...,42.40,NaN,NaN,NaN,0.002364,NaN,ABBN,Abb Ltd,2024-03-15,7.979691e+10
2,ABBN,2024-03-14,42.31,NaN,0.26,0.006183,42.15,42.09,42.72,5325771.0,...,42.32,NaN,NaN,NaN,-0.002123,NaN,ABBN,Abb Ltd,2024-03-14,7.962753e+10
3,ABBN,2024-03-13,42.05,NaN,0.53,0.012765,41.69,41.50,42.08,4401158.0,...,42.06,NaN,NaN,NaN,-0.006145,NaN,ABBN,Abb Ltd,2024-03-13,7.913821e+10
4,ABBN,2024-03-12,41.52,NaN,0.57,0.013919,41.02,40.69,41.52,4394509.0,...,41.52,NaN,NaN,NaN,-0.012604,NaN,ABBN,Abb Ltd,2024-03-12,7.814075e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
880,VLRT,2023-12-29,12.70,NaN,0.00,0.000000,12.70,12.70,12.70,200.0,...,12.70,NaN,NaN,NaN,0.016000,NaN,VLRT,Valartis Group AG,2023-12-29,3.956759e+07
881,VLRT,2023-12-28,12.70,NaN,-0.05,-0.003922,12.70,12.70,12.70,1668.0,...,12.70,NaN,NaN,NaN,0.000000,NaN,VLRT,Valartis Group AG,2023-12-28,3.977460e+07
882,VLRT,2023-12-27,12.75,NaN,0.30,0.024096,12.75,12.75,12.75,100.0,...,12.75,NaN,NaN,NaN,0.003937,NaN,VLRT,Valartis Group AG,2023-12-27,3.986026e+07
883,VLRT,2023-12-22,12.45,NaN,-1.05,-0.077778,12.45,12.45,12.45,580.0,...,12.45,NaN,NaN,NaN,-0.023529,NaN,VLRT,Valartis Group AG,2023-12-22,3.889509e+07


## Data Cleaning

We remove unnecessary columns in both dataframes before saving them to [`processed`](<..\data\processed>).

In [6]:
# drop columns
total_df.drop(columns=['Adjusted Close', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 12', 'Identifier', 'Exchange Date'], inplace=True)
sum_df.drop(columns=['Identifier'], inplace=True)

# reorder columns
cols = ['Ticker'] + ['Company Name'] + ['Date'] + [col for col in total_df if col not in ['Ticker', 'Company Name', 'Date']]
total_df = total_df[cols]

cols = ['Ticker'] + [col for col in sum_df if col != 'Ticker']
sum_df = sum_df[cols]

total_df.to_csv(os.path.join('..', 'data', 'processed', 'merged_stats.csv'), index=False)
sum_df.to_csv(os.path.join('..', 'data', 'processed', 'summary_data.csv'), index=False)